# 데이터 다운로드
https://gist.github.com/ktisha/c21e73a1bd1700294ef790c56c8aec1f

In [1]:
import tensorflow as tf
print(tf.__version__)
import tensorflow.keras.utils as utils
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
import numpy as np

np.random.seed(5)

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


2.0.0-alpha0


In [2]:
dataset = np.loadtxt("pima-indians-diabetes.csv", delimiter=",")

In [3]:
x_train = dataset[:700,0:8]
y_train = dataset[:700,8]
x_test = dataset[700:,0:8]
y_test = dataset[700:,8]

In [4]:
model = Sequential()
model.add(Dense(12, input_dim=8, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [5]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [8]:
hist = model.fit(x_train, y_train, epochs=1500, batch_size=64)

Epoch 1/1500
700/700 [==============================] - 0s 14us/sample - loss: 0.3829 - accuracy: 0.8229
Epoch 2/1500
700/700 [==============================] - 0s 15us/sample - loss: 0.3843 - accuracy: 0.8286
Epoch 3/1500
700/700 [==============================] - 0s 15us/sample - loss: 0.3845 - accuracy: 0.8229
Epoch 4/1500
700/700 [==============================] - 0s 14us/sample - loss: 0.3851 - accuracy: 0.8257
Epoch 5/1500
700/700 [==============================] - 0s 15us/sample - loss: 0.3853 - accuracy: 0.8329
Epoch 6/1500
700/700 [==============================] - 0s 15us/sample - loss: 0.3898 - accuracy: 0.8114
Epoch 7/1500
700/700 [==============================] - 0s 13us/sample - loss: 0.3824 - accuracy: 0.8271
Epoch 8/1500
700/700 [==============================] - 0s 15us/sample - loss: 0.3848 - accuracy: 0.8314
Epoch 9/1500
700/700 [==============================] - 0s 12us/sample - loss: 0.3860 - accuracy: 0.8314
Epoch 10/1500
700/700 [==============================] 

In [9]:
scores = model.evaluate(x_test, y_test)
print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))

68/68 [==============================] - 0s 29us/sample - loss: 0.9790 - accuracy: 0.7500
accuracy: 75.00%


# 컨볼루션 레이어 모델 만들어보기

In [24]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# 랜덤시드 고정시키기
np.random.seed(3)

# 1. 데이터 생성하기
train_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        'handwriting_shape/train',
        target_size=(24, 24),
        batch_size=3,
        class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
        'handwriting_shape/test',
        target_size=(24, 24),    
        batch_size=3,
        class_mode='categorical')

# 2. 모델 구성하기
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(24,24,3)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(3, activation='softmax'))

# 3. 모델 학습과정 설정하기
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 4. 모델 학습시키기
model.fit_generator(
        train_generator,
        steps_per_epoch=15,
        epochs=50,
        validation_data=test_generator,
        validation_steps=5)

# 5. 모델 평가하기
print("-- Evaluate --")
scores = model.evaluate_generator(test_generator, steps=5)
print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))

# 6. 모델 사용하기
print("-- Predict --")
output = model.predict_generator(test_generator, steps=5)
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
print(test_generator.class_indices)
print(output)


Found 45 images belonging to 3 classes.
Found 15 images belonging to 3 classes.
Epoch 1/50
15/15 [==============================] - 1s 46ms/step - loss: 1.2271 - accuracy: 0.4222 - val_loss: 0.7527 - val_accuracy: 0.8667
Epoch 2/50
15/15 [==============================] - 0s 13ms/step - loss: 0.3404 - accuracy: 0.9778 - val_loss: 0.1572 - val_accuracy: 0.9333
Epoch 3/50
15/15 [==============================] - 0s 13ms/step - loss: 0.0473 - accuracy: 1.0000 - val_loss: 0.0314 - val_accuracy: 1.0000
Epoch 4/50
15/15 [==============================] - 0s 13ms/step - loss: 0.0065 - accuracy: 1.0000 - val_loss: 0.0234 - val_accuracy: 1.0000
Epoch 5/50
15/15 [==============================] - 0s 13ms/step - loss: 9.1471e-04 - accuracy: 1.0000 - val_loss: 0.0164 - val_accuracy: 1.0000
Epoch 6/50
15/15 [==============================] - 0s 13ms/step - loss: 6.3152e-04 - accuracy: 1.0000 - val_loss: 0.0142 - val_accuracy: 1.0000
Epoch 7/50
15/15 [==============================] - 0s 13ms/step -

--- 
# 데이터 부풀리기

## 기존 결과 보기


In [27]:
# 랜덤시드 고정시키기
np.random.seed(3)

# 데이터셋 불러오기
train_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        'handwriting_shape/train',
        target_size=(24, 24),
        batch_size=3,
        class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
        'handwriting_shape/test_new',
        target_size=(24, 24),    
        batch_size=3,
        class_mode='categorical')


# 모델 구성하기
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(24,24,3)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(3, activation='softmax'))

# 모델 엮기
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 모델 학습시키기
model.fit_generator(
        train_generator,
        steps_per_epoch=15,
        epochs=200,
        validation_data=test_generator,
        validation_steps=5)

# 모델 평가하기
print("-- Evaluate --")

scores = model.evaluate_generator(
            test_generator, 
            steps = 5)

print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))

# 모델 예측하기
print("-- Predict --")

output = model.predict_generator(
            test_generator, 
            steps = 5)

np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})

print(output)

Found 45 images belonging to 3 classes.
Found 15 images belonging to 3 classes.
Epoch 1/200
15/15 [==============================] - 1s 47ms/step - loss: 1.1062 - accuracy: 0.5556 - val_loss: 1.1842 - val_accuracy: 0.3333
Epoch 2/200
15/15 [==============================] - 0s 13ms/step - loss: 0.2421 - accuracy: 1.0000 - val_loss: 2.5127 - val_accuracy: 0.3333
Epoch 3/200
15/15 [==============================] - 0s 13ms/step - loss: 0.0165 - accuracy: 1.0000 - val_loss: 3.3611 - val_accuracy: 0.3333
Epoch 4/200
15/15 [==============================] - 0s 13ms/step - loss: 0.0017 - accuracy: 1.0000 - val_loss: 4.0455 - val_accuracy: 0.3333
Epoch 5/200
15/15 [==============================] - 0s 13ms/step - loss: 5.0433e-04 - accuracy: 1.0000 - val_loss: 4.3319 - val_accuracy: 0.3333
Epoch 6/200
15/15 [==============================] - 0s 13ms/step - loss: 2.9941e-04 - accuracy: 1.0000 - val_loss: 4.4936 - val_accuracy: 0.3333
Epoch 7/200
15/15 [==============================] - 0s 13ms

## 데이터 부풀리기

In [29]:
train_datagen = ImageDataGenerator(rescale=1./255, 
                                   rotation_range=10,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.7,
                                   zoom_range=[0.9, 2.2],
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                   fill_mode='nearest')

train_generator = train_datagen.flow_from_directory(
        'handwriting_shape/train',
        target_size=(24, 24),
        batch_size=3,
        class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
        'handwriting_shape/test_new',
        target_size=(24, 24),    
        batch_size=3,
        class_mode='categorical')


model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(24,24,3)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(3, activation='softmax'))

# 모델 엮기
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 모델 학습시키기
model.fit_generator(
        train_generator,
        steps_per_epoch=15 * 100,
        epochs=200,
        validation_data=test_generator,
        validation_steps=5)

# 모델 평가하기
print("-- Evaluate --")

scores = model.evaluate_generator(
            test_generator, 
            steps = 5)

print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))

# 모델 예측하기
print("-- Predict --")

output = model.predict_generator(
            test_generator, 
            steps = 5)

np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})

print(output)

Found 45 images belonging to 3 classes.
Found 15 images belonging to 3 classes.
Epoch 1/200
1500/1500 [==============================] - 14s 9ms/step - loss: 0.2116 - accuracy: 0.9144 - val_loss: 1.7397 - val_accuracy: 0.7333
Epoch 2/200
1500/1500 [==============================] - 13s 9ms/step - loss: 0.0397 - accuracy: 0.9867 - val_loss: 2.4508 - val_accuracy: 0.8000
Epoch 3/200
1500/1500 [==============================] - 13s 9ms/step - loss: 0.0244 - accuracy: 0.9916 - val_loss: 1.9214 - val_accuracy: 0.8000
Epoch 4/200
1500/1500 [==============================] - 13s 9ms/step - loss: 0.0225 - accuracy: 0.9936 - val_loss: 2.3749 - val_accuracy: 0.8000
Epoch 5/200
1500/1500 [==============================] - 13s 9ms/step - loss: 0.0201 - accuracy: 0.9936 - val_loss: 1.6965 - val_accuracy: 0.7333
Epoch 6/200
1500/1500 [==============================] - 13s 9ms/step - loss: 0.0206 - accuracy: 0.9942 - val_loss: 3.0659 - val_accuracy: 0.8000
Epoch 7/200
1500/1500 [=====================

KeyboardInterrupt: 